In [2]:
# 0. API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API 키 정보 로드
load_dotenv(dotenv_path='C:/Users/user/Desktop/RAG-Implementation/CSV RAG implementation/.env')
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [3]:
'''1. INDEXING'''
# 1. 문서 로드: pandas로 CSV 직접 읽기
# RAG 방식은 일부 문서만 검색하므로 집계 질문에 부정확함
# → pandas로 전체 데이터를 먼저 집계한 뒤, 그 결과를 LLM에 전달하는 하이브리드 방식 사용
import pandas as pd

df = pd.read_csv(
    'C:/Users/user/Downloads/한국동서발전(주)_대기권 밖 일사량_20240125.csv',
    encoding='cp949'
)

# 시간 컬럼 추출 (집계에 활용)
df['시간'] = pd.to_datetime(df['일자'], format='%m-%d %H:%M').dt.hour
df['월일'] = df['일자'].str[:5]  # ex) '01-01'

# 데이터 확인
print(f"총 행 수: {len(df)}")
print(f"지역 목록: {df['지역'].unique().tolist()}")
print()
print(df.head())

총 행 수: 78840
지역 목록: ['광주', '대구', '대전', '부산', '서울', '원주', '전주', '제주', '청주']

            일자  지역       위도        경도  대기권 밖 일사량  시간     월일
0  01-01 00:00  광주  35.1729  126.8916          0   0  01-01
1  01-01 01:00  광주  35.1729  126.8916          0   1  01-01
2  01-01 02:00  광주  35.1729  126.8916          0   2  01-01
3  01-01 03:00  광주  35.1729  126.8916          0   3  01-01
4  01-01 04:00  광주  35.1729  126.8916          0   4  01-01


In [4]:
'''2. 집계 함수 정의'''
# pandas로 정확한 데이터 집계를 수행하는 함수들
# 질문 유형에 따라 적절한 함수를 선택해서 context를 생성함

def get_hourly_avg(region: str) -> str:
    """지역별 시간대 평균 일사량 집계"""
    result = df[df['지역'] == region].groupby('시간')['대기권 밖 일사량'].mean()
    lines = [f"{region} 지역 시간대별 연평균 대기권 밖 일사량:"]
    for hour, val in result.items():
        lines.append(f"  {hour:02d}시: {val:,.0f}")
    return "\n".join(lines)


def get_daily_avg_by_region(region: str, month_day: str) -> str:
    """특정 지역, 특정 날짜의 시간대별 일사량 집계 (ex. month_day='01-01')"""
    filtered = df[(df['지역'] == region) & (df['월일'] == month_day)]
    lines = [f"{region} {month_day} 시간대별 대기권 밖 일사량:"]
    for _, row in filtered.iterrows():
        lines.append(f"  {row['일자']}: {row['대기권 밖 일사량']:,}")
    avg = filtered['대기권 밖 일사량'].mean()
    lines.append(f"  → 해당일 평균: {avg:,.1f}")
    return "\n".join(lines)


def get_region_comparison() -> str:
    """전체 지역 연평균 일사량 비교"""
    result = df.groupby('지역')['대기권 밖 일사량'].mean().sort_values(ascending=False)
    lines = ["지역별 연평균 대기권 밖 일사량 (높은 순):"]
    for region, val in result.items():
        lines.append(f"  {region}: {val:,.0f}")
    return "\n".join(lines)


def get_peak_hour_by_region(region: str) -> str:
    """지역별 일사량이 가장 높은 시간대"""
    result = df[df['지역'] == region].groupby('시간')['대기권 밖 일사량'].mean()
    peak_hour = result.idxmax()
    peak_val = result.max()
    lines = [f"{region} 지역 시간대별 연평균 일사량:"] 
    for hour, val in result.items():
        marker = " ← 최고" if hour == peak_hour else ""
        lines.append(f"  {hour:02d}시: {val:,.0f}{marker}")
    lines.append(f"\n→ 가장 높은 시간대: {peak_hour}시 (평균 {peak_val:,.0f})")
    return "\n".join(lines)

In [10]:
'''3. LLM 설정'''
# HuggingFace LLM 모델 설정
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

repo_id = "google/gemma-2-9b-it"

llm_endpoint = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_new_tokens=512,
    temperature=0.1,
    huggingfacehub_api_token=hf_token
)

chat_llm = ChatHuggingFace(llm=llm_endpoint)

In [11]:
'''4. 프롬프트 및 체인 설정'''
# pandas 집계 결과(context)를 받아서 LLM이 자연어로 설명하는 체인
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """당신은 일사량 데이터를 분석해주는 AI 어시스턴트입니다.
아래에 제공된 집계 데이터를 바탕으로 질문에 친절하게 답변해주세요.
숫자는 그대로 활용하되, 자연스러운 한국어 문장으로 설명해주세요.
답변은 반드시 한국어로 작성하세요.

#집계 데이터 (pandas로 전체 데이터를 집계한 정확한 값):
{context}

#질문:
{question}

#답변:"""

prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

# LLM 체인 (retriever 없이 context를 직접 전달)
llm_chain = prompt | chat_llm | output_parser

In [12]:
'''5. 실행 함수'''
# 집계 함수로 context를 만들고 LLM에 전달하는 통합 함수

def ask(question: str, context: str):
    """집계된 context와 질문을 LLM에 전달해서 자연어 답변 생성"""
    response = llm_chain.invoke({"context": context, "question": question})
    print(f"질문: {question}")
    print("-" * 40)
    print(f"답변: {response}")
    return response

In [13]:
# 테스트 1: 서울 일사량이 가장 높은 시간대
context = get_peak_hour_by_region("서울")
question = "서울 평균 일사량이 가장 높은 시간대를 알려줘"

ask(question, context)

질문: 서울 평균 일사량이 가장 높은 시간대를 알려줘
----------------------------------------
답변: 서울에서 평균 일사량이 가장 높은 시간대는 **오후 12시**입니다. ☀️  





'서울에서 평균 일사량이 가장 높은 시간대는 **오후 12시**입니다. ☀️  \n\n\n'

In [14]:
# 테스트 2: 광주 1월 1일 시간대별 일사량 및 평균
context = get_daily_avg_by_region("광주", "01-01")
question = "광주 1월 1일의 시간대별 일사량과 평균을 알려줘"

ask(question, context)

질문: 광주 1월 1일의 시간대별 일사량과 평균을 알려줘
----------------------------------------
답변: 광주 1월 1일의 시간대별 일사량은 아래와 같습니다. 

* 00:00 ~ 06:00은 0였고, 07:00부터 18:00까지 일사량이 증가하여 12:00과 13:00에 가장 높은 2,632,916으로 측정되었습니다. 
* 이후 18:00부터는 다시 0으로 감소했습니다.

광주 1월 1일의 전체 평균 일사량은 697,657입니다. 





'광주 1월 1일의 시간대별 일사량은 아래와 같습니다. \n\n* 00:00 ~ 06:00은 0였고, 07:00부터 18:00까지 일사량이 증가하여 12:00과 13:00에 가장 높은 2,632,916으로 측정되었습니다. \n* 이후 18:00부터는 다시 0으로 감소했습니다.\n\n광주 1월 1일의 전체 평균 일사량은 697,657입니다. \n\n\n'

In [15]:
# 테스트 3: 어떤 지역이 가장 많은 일사량을 받는지
context = get_region_comparison()
question = "어떤 지역이 연평균 일사량이 가장 높아? 이유도 추측해줘"

ask(question, context)

질문: 어떤 지역이 연평균 일사량이 가장 높아? 이유도 추측해줘
----------------------------------------
답변: 연평균 대기권 밖 일사량이 가장 높은 지역은 제주입니다. 제주는 1,274,852 kWh로, 다른 지역에 비해 꽤 높은 일사량을 기록했습니다. 

제주도가 다른 지역보다 일사량이 높은 이유는 섬의 위치와 기후 특성 때문으로 추측됩니다. 제주도는 남쪽에 위치하여 북쪽 지역에 비해 햇빛을 더 많이 받을 수 있습니다. 또한, 섬 주변의 바다는 해양의 영향으로 기온 변화가 적고 맑은 날씨가 많아 햇빛이 덜 가려지기 때문일 수 있습니다.  






'연평균 대기권 밖 일사량이 가장 높은 지역은 제주입니다. 제주는 1,274,852 kWh로, 다른 지역에 비해 꽤 높은 일사량을 기록했습니다. \n\n제주도가 다른 지역보다 일사량이 높은 이유는 섬의 위치와 기후 특성 때문으로 추측됩니다. 제주도는 남쪽에 위치하여 북쪽 지역에 비해 햇빛을 더 많이 받을 수 있습니다. 또한, 섬 주변의 바다는 해양의 영향으로 기온 변화가 적고 맑은 날씨가 많아 햇빛이 덜 가려지기 때문일 수 있습니다.  \n\n\n\n'

In [16]:
# 테스트 4: 특정 지역 시간대별 평균 전체 조회
context = get_hourly_avg("제주")
question = "제주 지역의 시간대별 평균 일사량 패턴을 설명해줘"

ask(question, context)

질문: 제주 지역의 시간대별 평균 일사량 패턴을 설명해줘
----------------------------------------
답변: 제주 지역의 일사량은 낮 시간대에 가장 높고, 밤 시간대에는 0입니다. 

구체적으로, 오전 6시부터 시작하여 오후 12시까지 일사량이 꾸준히 증가하며, 오후 12시부터 오후 3시까지는 389만 1,619 W/m²로 가장 높은 값을 보입니다. 이후 오후 3시부터 오후 6시까지는 점차 감소하며, 저녁 6시부터는 다시 감소하여 밤 9시까지는 0 W/m²로 떨어집니다. 





'제주 지역의 일사량은 낮 시간대에 가장 높고, 밤 시간대에는 0입니다. \n\n구체적으로, 오전 6시부터 시작하여 오후 12시까지 일사량이 꾸준히 증가하며, 오후 12시부터 오후 3시까지는 389만 1,619 W/m²로 가장 높은 값을 보입니다. 이후 오후 3시부터 오후 6시까지는 점차 감소하며, 저녁 6시부터는 다시 감소하여 밤 9시까지는 0 W/m²로 떨어집니다. \n\n\n'